In [75]:
import matplotlib.pyplot as plt
import pandas as pd

#import data
price_data = pd.read_csv('Input Data/Day Data/day_data_csv.csv')
price_data.dtypes

ema(4)                     float64
high low span              float64
high above ema             float64
close above ema            float64
close relative to high     float64
volume change              float64
volume relative to span    float64
to buy                       int64
dtype: object

In [76]:
#split into x and y 
X = price_data.drop("to buy", axis=1)
y = price_data["to buy"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=0)

In [77]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [78]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=33)


gbc = VotingClassifier(estimators=[('Decision Tree', dtc), ('Nearest Neighbors', knn )])

gbc = gbc.fit(X_train, y_train)

training_accur = gbc.score(X_train_scaled, y_train)
testing_accur = gbc.score(X_test_scaled, y_test) 

print(f"Training Accuracy Score: {training_accur}")
print(f"Testing Accuracy Score: {testing_accur}")


Training Accuracy Score: 0.08358448864777979
Testing Accuracy Score: 0.08197709463532249


In [79]:
importances = gbc.feature_importances_
feature_names = ['ema (4)','high low span','high above ema','close above ema','close relative to high','volume change','volume relative to span']

sorted(zip(gbc.feature_importances_, feature_names), reverse=True)

AttributeError: 'VotingClassifier' object has no attribute 'feature_importances_'

In [80]:
# Use our model to make predictions and generate confusion matrix
y_test_predict = gbc.predict(X_test_scaled)
y_train_predict = gbc.predict(X_train_scaled)

#price_data["Prediction"]=predict_all
#price_data.to_csv('predicted_price_data.csv')

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_test_predict))
print(confusion_matrix(y_train, y_train_predict))

[[ 62  83 115 ...   0   0   0]
 [ 71 101 123 ...   0   0   0]
 [ 66 107 132 ...   0   0   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[114 194 245 ...   0   0   0]
 [133 227 244 ...   0   0   0]
 [121 227 266 ...   0   0   0]
 ...
 [  0   1   1 ...   0   0   0]
 [  0   1   2 ...   0   0   0]
 [  0   1   2 ...   0   0   0]]


In [81]:
#Summary of model performance
from sklearn.metrics import mean_squared_error, accuracy_score

training_score = accuracy_score(y_train, y_train_predict)
testing_score = accuracy_score(y_test, y_test_predict) 

print(f"Training Accuracy: {training_score}")
print(f"Testing Accuracy: {testing_score}")

training_mse = mean_squared_error(y_train, y_train_predict)
testing_mse = mean_squared_error(y_test, y_test_predict)

print(f"Training Mean Squared Error (MSE): {training_mse}")
print(f"Testing Mean Squared Error (MSE): {testing_mse}")


Training Accuracy: 0.08358448864777979
Testing Accuracy: 0.08197709463532249
Training Mean Squared Error (MSE): 85.99336949969862
Testing Mean Squared Error (MSE): 88.09905565601768
